# README

This readme file contains documentation for combining and processing a dataset related to Wikipedia page traffic and analyzing it.


Below are the steps involved
## Gather the data
We need to collect monthly data from the below
There are 5 things here
Legacy Pagecounts API
1. Desktop data
2. Mobile data

Pageviews API
1. Desktop data
2. Mobile web data
3. Mobile app traffic data

### Imports
Before we do any processing, we need to import all required libraries for python. These are available on PyPi repository

In [115]:
import json
import requests
import pandas as pd
from datetime import datetime

### Constants and common definitions
The below code will declare constants like endpoints, headers and define the access points supported by the different endpoints

In [70]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/en.wikipedia.org/{access}/monthly/{start}/{end}'
endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/en.wikipedia.org/{access}/{agent}/monthly/{start}/{end}'
headers = {
    'User-Agent': 'https://github.com/vikrantb',
    'From': 'vikrantb@uw.edu'
}

legacy_access_points = ['desktop-site', 'mobile-site']
page_view_access_points = ['desktop', 'mobile-app', 'mobile-web']

### Common helper functions

Below are some helper functions that will be used throughout. With an explanation

#### api_call
This method makes a call to the endpoint provided with the parameters provided. It also has an option to directly extract the items

#### get_wiki_df
This method returns the pagecount/pageviews information in a dataframe format

#### get_from_legacy
This method is a wrapper method on top of api_call specifically for legacy endpoint

#### get_from_page_views
This method is a wrapper method on top of api_call specifically for pageviews endpoint

#### convert_to_simple_date
This is a simple helper method that helps convert a string date of the format `YYYYMMDDHH` into a simpler date by ignoring the day and hour part

In [277]:
def api_call(endpoint,parameters, extract_items=False):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response['items'] if extract_items else response

def get_from_legacy(access, start, end, save_file=False):
    params = {"project" : "en.wikipedia.org",
             "access" : access,
             "granularity" : "monthly",
             "start" : "{start}0100".format(start=start),
             "end" : "{end}0100".format(end=end)
            }
    data = api_call(endpoint=endpoint_legacy, parameters=params)
    file_name="pagecounts_{access}_{start}-{end}.json".format(access=access, start=start, end=end)
    with open(file_name, 'w') as f:
        json.dump(data, f)


def get_from_page_views(access, start, end, save_file=False):
    params = {"project" : "en.wikipedia.org",
             "access" : access,
             "agent" : "user",
             "granularity" : "monthly",
             "start" : "{start}0100".format(start=start),
             "end" : "{end}0100".format(end=end)
            }
    data = api_call(endpoint=endpoint_pageviews, parameters=params)
    file_name="pageviews_{access}_{start}-{end}.json".format(access=access, start=start, end=end)
    with open(file_name, 'w') as f:
        json.dump(data, f)
        
def convert_to_simple_date(row):
    yymm = str(row['timestamp'])
    year=yymm[:4]
    month=yymm[4:6]
    row['year'] = year
    row['month'] = month
    if 'views' in row:
        row['num_views'] = row['views']  
    if 'count' in row:
        row['num_views'] = row['count']  
    return row


def get_wiki_df(file_path):
    wiki_json = ''
    with open(file_path) as f:
        wiki_json = str(json.load(f)['items']).replace("\'", "\"")
    df = pd.read_json(wiki_json, orient='records', convert_dates=False)
    df['timestamp'] = df['timestamp'].apply(str)
    df['year'] = df['timestamp'].apply(lambda x: x[:4])
    df['month'] = df['timestamp'].apply(lambda x: x[4:6])
    df['num_views'] = df['views'] if 'views' in df.columns else df['count']
    return df


## Step 1: Gathering the data

In this step we will gather the wikipedia pagecount data per month level.
As noted below, the data is available in legacy endpoint from January 2008 to July 2016
As noted below, the data is available in pageviews endpoint from July 2015 2008 to September 2020.

There is an overlap for data from July 2015 to July 2016

After execution of the below step, there will be 5 json files created as below

- pagecounts_desktop-site_200801-201607.json - desktop site information from legacy api from January 2008 to July 2016
- pagecounts_mobile-site_200801-201607.json - mobile site information from legacy api from January 2008 to July 2016
- pageviews_desktop_201507-202009.json - desktop information from pageviews api from July 2015 to September 2020
- pageviews_mobile-app_201507-202009.json - mobile app information from pageviews api from July 2015 to September 2020
- pageviews_mobile-web_201507-202009.json - mobile web information from pageviews api from July 2015 to September 2020

In [196]:
legacy_start_date='200801'
legacy_end_date='201607'
for access_type in legacy_access_points:
    get_from_legacy(access=access_type, start=legacy_start_date,end=legacy_end_date, save_file=True)

legacy_start_date='201507'
legacy_end_date='202009'
for access_type in page_view_access_points:
    get_from_page_views(access=access_type, start=legacy_start_date,end=legacy_end_date, save_file=True)

In [279]:
pagecounts_mobile_df = get_wiki_df(pagecounts_mobile_fp)

In [280]:
pagecounts_mobile_df.head(30)

,project,access-site,granularity,timestamp,count,year,month,num_views
0,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
1,en.wikipedia,mobile-site,monthly,2014110100,3027489668,2014,11,3027489668
2,en.wikipedia,mobile-site,monthly,2014120100,3278950021,2014,12,3278950021
3,en.wikipedia,mobile-site,monthly,2015010100,3485302091,2015,01,3485302091
4,en.wikipedia,mobile-site,monthly,2015020100,3091534479,2015,02,3091534479
5,en.wikipedia,mobile-site,monthly,2015030100,3330832588,2015,03,3330832588
6,en.wikipedia,mobile-site,monthly,2015040100,3222089917,2015,04,3222089917
7,en.wikipedia,mobile-site,monthly,2015050100,3334069483,2015,05,3334069483
8,en.wikipedia,mobile-site,monthly,2015060100,3038162463,2015,06,3038162463
9,en.wikipedia,mobile-site,monthly,2015070100,3254472695,2015,07,3254472695


## Step 2: Processing the data
The objective of this step is to combine data from all the json files collected in the gathering step into a single csv file upon which we will run analysis.

For the pageviews api, we will sum the counts from mobile-app and mobile-web to create a single value for mobile traffic.

For all the data, we will be only using the `YYYYMM` part of the timestamp and ignoring the `DDHH` part

In [253]:
pagecounts_desktop_fp = 'pagecounts_desktop-site_200801-201607.json'
pagecounts_mobile_fp = 'pagecounts_mobile-site_200801-201607.json'
pageviews_desktop_fp = 'pageviews_desktop_201507-202009.json'
pageviews_mobile_app_fp = 'pageviews_mobile-app_201507-202009.json'
pageviews_mobile_web_fp = 'pageviews_mobile-web_201507-202009.json'

pagecounts_desktop_df = get_wiki_df(pagecounts_desktop_fp)
pagecounts_mobile_df = get_wiki_df(pagecounts_mobile_fp)
pageviews_desktop_df = get_wiki_df(pageviews_desktop_fp)
pageviews_mobile_app_df = get_wiki_df(pageviews_mobile_app_fp)
pageviews_mobile_web_df = get_wiki_df(pageviews_mobile_web_fp)

[{"project": "en.wikipedia", "access-site": "desktop-site", "granularity": "monthly", "timestamp": "2008010100", "count": 4930902570}, {"project": "en.wikipedia", "access-site": "desktop-site", "granularity": "monthly", "timestamp": "2008020100", "count": 4818393763}, {"project": "en.wikipedia", "access-site": "desktop-site", "granularity": "monthly", "timestamp": "2008030100", "count": 4955405809}, {"project": "en.wikipedia", "access-site": "desktop-site", "granularity": "monthly", "timestamp": "2008040100", "count": 5159162183}, {"project": "en.wikipedia", "access-site": "desktop-site", "granularity": "monthly", "timestamp": "2008050100", "count": 5584691092}, {"project": "en.wikipedia", "access-site": "desktop-site", "granularity": "monthly", "timestamp": "2008060100", "count": 5712104279}, {"project": "en.wikipedia", "access-site": "desktop-site", "granularity": "monthly", "timestamp": "2008070100", "count": 5306302874}, {"project": "en.wikipedia", "access-site": "desktop-site", "g

In [249]:
pagecounts_mobile_df.head(30)

,project,access-site,granularity,timestamp,count,year,month,num_views
0,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
1,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
2,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
3,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
4,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
5,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
6,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
7,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
8,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685
9,en.wikipedia,mobile-site,monthly,2033-10-28 08:15:00,3091546685,2033,-1,3091546685


In [237]:
pageviews_mobile_df = pd.concat([pageviews_mobile_app_df,pageviews_mobile_web_df])
pageviews_mobile_df = pageviews_mobile_df.groupby(['year', 'month']).sum().reset_index()
pagecount_all_df = pd.concat([pagecounts_desktop_df, pagecounts_mobile_df])
pagecount_all_df = pagecount_all_df.groupby(['year', 'month']).sum().reset_index()

pageview_all_df = pd.concat([pageviews_desktop_df, pageviews_mobile_df])
pageview_all_df = pageview_all_df.groupby(['year', 'month']).sum().reset_index()

,project,access-site,granularity,timestamp,count,year,month,num_views
0,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
1,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
2,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
3,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
4,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
5,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
6,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
7,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
8,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685
9,en.wikipedia,mobile-site,monthly,2014100100,3091546685,2014,10,3091546685


In [211]:
pagecount_all_of_all_df = pd.concat([pagecount_all_df, pageview_all_df])[['year', 'month']]
pagecount_all_of_all_df.head(30)

,year,month
0,2008,01
1,2014,10
0,2015,07


In [201]:
combined_df.head(3)

,year,month,num_views
0,2008,01,4930902570
1,2008,01,4930902570
2,2008,01,4930902570
